In [3]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
from collections import Counter
import matplotlib
from matplotlib.colors import ListedColormap
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

current_palette = sns.color_palette()
cmap = ListedColormap(sns.color_palette(current_palette).as_hex())

sns.set(rc={'figure.figsize':(10,4)}, font_scale=1.0, style='whitegrid', font='CMU Sans Serif')
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

def save_fig(ax, folder, suffix, format="pdf"):
    fig = ax.get_figure()
    fig.tight_layout()

    fig.savefig(folder + suffix + "." + format)
    fig.clear()

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
locations = pd.read_csv(os.path.join("..", "eu_cities.csv"))
locations = locations[["city_ascii", "lng", "lat"]]
locations.rename(columns = {"city_ascii": "Client", "lng": "Longitude", "lat": "Latitude"}, inplace=True)

In [6]:
RESULTS_DIR = os.path.join("..", "results")
FILENAME = "results-"

In [7]:
df_paper = pd.DataFrame()

for experiment in ["cloud", "sat"]:
    for run in ["3"]: #for run in ["1", "2", "3"]:
        NAME = FILENAME + experiment + "-" + run
        files = os.listdir(os.path.join(".", RESULTS_DIR, NAME))

        df_experiment = pd.DataFrame()

        for f in files:
            if f.endswith("-history.csv"):
                client = f[2:-len(".ext4-history.csv")]
                # print(client)
                df_client = pd.read_csv(os.path.join(".", RESULTS_DIR, NAME, f))
                df_client.dropna(inplace=True)
                df_run = df_client[["Timestamp", "Type", "Name", "50%", "90%", "Total Median Response Time", "Total Average Response Time"]].copy()

                df_run["Client"] = client
                df_run["Experiment"] = experiment
                df_run["Run"] = run
                df_run["Latitude"] = locations.loc[locations["Client"] == client, "Latitude"].values[0]
                df_run["Longitude"] = locations.loc[locations["Client"] == client, "Longitude"].values[0]
                df_experiment = df_experiment.append(df_run)

        df_experiment["Timestamp"] = df_experiment["Timestamp"] - min(df_experiment["Timestamp"])
        start_time = max(df_experiment["Timestamp"]) - 1000
        df_experiment = df_experiment[ (df_experiment["Timestamp"] >  start_time) & (df_experiment["Timestamp"] < start_time+900) ]
        df_experiment["Timestamp"] = df_experiment["Timestamp"] - min(df_experiment["Timestamp"])

        df_paper = df_paper.append(df_experiment)

df_paper.head()


,Timestamp,Type,Name,50%,90%,Total Median Response Time,Total Average Response Time,Client,Experiment,Run,Latitude,Longitude
1683,0.0,GET,/,19000.0,19000.0,290.0,714.211111,Malaga,cloud,3,36.7194,-4.42
1684,0.0,POST,/,0.0,0.0,490.0,889.812500,Malaga,cloud,3,36.7194,-4.42
1685,0.0,GET,/auth/find-friends/,250.0,260.0,240.0,366.386792,Malaga,cloud,3,36.7194,-4.42
1686,0.0,GET,/auth/login/,0.0,0.0,110.0,110.000000,Malaga,cloud,3,36.7194,-4.42
1687,0.0,POST,/auth/login/,0.0,0.0,490.0,1156.125000,Malaga,cloud,3,36.7194,-4.42


In [8]:
map_df = df_paper[ ((df_paper["Name"] == "/public/") & (df_paper["Type"] == "GET")) | ((df_paper["Name"] == "/") & (df_paper["Type"] == "POST")) ].groupby(["Client", "Experiment", "Type", "Name"], as_index=False).median()
map_df

,Client,Experiment,Type,Name,Timestamp,50%,90%,Total Median Response Time,Total Average Response Time,Latitude,Longitude
0,ACoruna,cloud,GET,/public/,449.0,590.0,700.0,380.0,534.589744,43.3713,-8.4188
1,ACoruna,cloud,POST,/,449.0,0.0,0.0,840.0,1292.162791,43.3713,-8.4188
2,ACoruna,sat,GET,/public/,448.5,210.0,280.0,160.0,175.199700,43.3713,-8.4188
3,ACoruna,sat,POST,/,448.5,520.0,530.0,420.0,448.169697,43.3713,-8.4188
4,Alicante,cloud,GET,/public/,449.0,480.0,590.0,380.0,679.670659,38.3453,-0.4831
...,...,...,...,...,...,...,...,...,...,...,...
475,Zaragoza,sat,POST,/,448.5,720.0,720.0,720.0,817.734694,41.6483,-0.8830
476,Zurich,cloud,GET,/public/,449.0,390.0,530.0,290.0,630.344086,47.3786,8.5400
477,Zurich,cloud,POST,/,449.0,450.0,450.0,500.0,1068.200000,47.3786,8.5400
478,Zurich,sat,GET,/public/,448.5,290.0,515.0,220.0,310.168655,47.3786,8.5400


In [16]:
avg_cloud_get =  df_paper[ ((df_paper["Name"] == "/public/") & (df_paper["Type"] == "GET")) & (df_paper["Experiment"] == "cloud") ]["Total Average Response Time"].mean()
print("Avg cloud GET: %.1f" % avg_cloud_get)
avg_sat_get = df_paper[ ((df_paper["Name"] == "/public/") & (df_paper["Type"] == "GET")) & (df_paper["Experiment"] == "sat") ]["Total Average Response Time"].mean()
print("Avg sat GET: %.1f" % avg_sat_get)

avg_cloud_post = df_paper[ ((df_paper["Name"] == "/") & (df_paper["Type"] == "POST")) & (df_paper["Experiment"] == "cloud") ]["Total Average Response Time"].mean()
print("Avg cloud POST: %.1f" % avg_cloud_post)
avg_sat_post = df_paper[ ((df_paper["Name"] == "/") & (df_paper["Type"] == "POST")) & (df_paper["Experiment"] == "sat") ]["Total Average Response Time"].mean()
print("Avg sat POST: %.1f" % avg_sat_post)

get_improvement = (avg_cloud_get - avg_sat_get) / avg_cloud_get
print("GET improvement: %.1f%%" % (get_improvement*100))
post_improvement = (avg_cloud_post - avg_sat_post) / avg_cloud_post
print("POST improvement: %.1f%%" % (post_improvement*100))

median_cloud_get =  df_paper[ ((df_paper["Name"] == "/public/") & (df_paper["Type"] == "GET")) & (df_paper["Experiment"] == "cloud") ]["Total Median Response Time"].median()
print("median cloud GET: %.1f" % median_cloud_get)
median_sat_get = df_paper[ ((df_paper["Name"] == "/public/") & (df_paper["Type"] == "GET")) & (df_paper["Experiment"] == "sat") ]["Total Median Response Time"].median()
print("mediang sat GET: %.1f" % median_sat_get)

median_cloud_post = df_paper[ ((df_paper["Name"] == "/") & (df_paper["Type"] == "POST")) & (df_paper["Experiment"] == "cloud") ]["Total Median Response Time"].median()
print("median cloud POST: %.1f" % median_cloud_post)
median_sat_post = df_paper[ ((df_paper["Name"] == "/") & (df_paper["Type"] == "POST")) & (df_paper["Experiment"] == "sat") ]["Total Median Response Time"].median()
print("median sat POST: %.1f" % median_sat_post)

med_get_improvement = (median_cloud_get - median_sat_get) / median_cloud_get
print("med GET improvement: %.1f%%" % (med_get_improvement*100))
med_post_improvement = (median_cloud_post - median_sat_post) / median_cloud_post
print("med POST improvement: %.1f%%" % (med_post_improvement*100))


Avg cloud GET: 565.4
Avg sat GET: 275.4
Avg cloud POST: 1107.3
Avg sat POST: 573.8
GET improvement: 51.3%
POST improvement: 48.2%
median cloud GET: 300.0
Avmediang sat GET: 180.0
median cloud POST: 580.0
median sat POST: 400.0
med GET improvement: 40.0%
med POST improvement: 31.0%
